# Neo4j Graph Data Science Starter Kit
This notebook acts as a simple starter kit for using the Neo4j GDS library from Python.
It contains code fragments to do the following:
1. Set up a connection to Neo4j and read/write data.
3. Create graph projections to run your algorithm on.
4. Run algorithms and stream back results to Neo4j.
5. Use the algorithm results as new features for your predictive models.

This example uses the Game of Thrones dataset as present in the Neo4j graph data science sandbox. You can get your own for free here:
https://sandbox.neo4j.com/login?usecase=graph-data-science

## 1. Setting up the Neo4j Driver
Enter your own Neo4j credentials here:

In [1]:
url = "bolt://34.201.68.240:33513"
user = "neo4j"
password = "chicken-nuggets" 

In [2]:
from neo4j import GraphDatabase
driver = GraphDatabase.driver(url, auth=(user, password))
neo4j = driver.session()

### Example - Reading Neo4j results using the driver

In [3]:
import pandas as pd

result = neo4j.run('MATCH (n:Person) RETURN n.name AS name, n.age as age LIMIT 10')
df = pd.DataFrame(result.data())
print(df)

                    name   age
0    Gunthor son of Gurn   NaN
1  High Septon (fat_one)   NaN
2        Jaime Lannister  39.0
3         Gregor Clegane  35.0
4            Andros Brax   NaN
5           Roose Bolton  45.0
6         Wylis Manderly  53.0
7          Medger Cerwyn   NaN
8       Harrion Karstark   NaN
9         Halys Hornwood   NaN


## 2. Creating a graph projection
As an example, we want to analyze which people are most influential using the PageRank algorithm.

First, create a graph projection `interactions` that contains only the pattern we are interested in: `(:Person)-[:INTERACTS]->(:Person)`. 

Then, go through the following steps:
- Check if we have enough memory to generate it.
- Check if the graph projection already exists, if so, delete it.
- Create the graph projection.


### Estimating the required size of the projection

In [4]:
# Run the Cypher query
result = neo4j.run("""
CALL gds.graph.create.cypher.estimate(
    'MATCH (p) WHERE p:Person RETURN id(p) as id',
    'MATCH (p)-[:INTERACTS]->(p2:Person) RETURN id(p) AS source, id(p2) AS target')
""")

# Print the results
row = result.single()
print("Estimated:", row['nodeCount'], "nodes,", row['relationshipCount'], "relationships,", row['requiredMemory']," memory required.")

Estimated: 2642 nodes, 16747 relationships, 333 KiB  memory required.


### Clear existing in-memory graphs (if they exist)


In [5]:
import pprint 

# This query drops the projected graph if it already exists, else it returns 'None'.
result = neo4j.run("""
CALL gds.graph.exists($name) YIELD exists
WHERE exists
CALL gds.graph.drop($name) YIELD graphName
RETURN graphName + " was dropped." as message
""", name = 'interactions-all-books')

# Print the results
pprint.pprint(result.data())

[{'message': 'interactions-all-books was dropped.'}]


### Creating the new graph projection

In [6]:
# Create a weighted Cypher projection graph of (Person)-[:INTERACTS]->(:Person)
result = neo4j.run("""
CALL gds.graph.create.cypher(
    'interactions-all-books',
    'MATCH (p) WHERE p:Person RETURN id(p) as id',
    'MATCH (p)-[i:INTERACTS]->(p2:Person) RETURN id(p) AS source, i.weight as weight, id(p2) AS target')
""")

# Print the results
row = result.single()
print(row['graphName'],"-", row['nodeCount'], "nodes,", row['relationshipCount'], "relationships,", row['createMillis']," ms to create the projection.")


interactions-all-books - 2166 nodes, 3907 relationships, 8  ms to create the projection.


## 3. Running graph algorithms
Now that we have our graph project, we're ready to run the algorithm!

As always, best practice is to first check if we have enough memory for running the algorithm.

In [7]:
result = neo4j.run("""
CALL gds.pageRank.stream.estimate('interactions-all-books',  { relationshipWeightProperty: 'weight' })
""")

print(result.single()['requiredMemory'], ' memory required to run the algorithm.')

84 KiB  memory required to run the algorithm.


### Run the algorithm (stream mode)
First, use 'stream' mode to inspect the results:

In [8]:
result = neo4j.run("""
CALL gds.pageRank.stream('interactions-all-books', { relationshipWeightProperty: 'weight'}) 
YIELD nodeId, score
RETURN gds.util.asNode(nodeId).name as character, score 
ORDER BY score DESC
""")

df = pd.DataFrame(result.data())
print(df)

              character      score
0      Tyrion Lannister  16.006423
1       Tywin Lannister   8.975134
2                 Varys   8.487427
3     Stannis Baratheon   8.152841
4         Theon Greyjoy   5.456874
...                 ...        ...
2161     Malaquo Maegyr   0.150000
2162              Morgo   0.150000
2163      Old Bill Bone   0.150000
2164               Scar   0.150000
2165      Shrouded Lord   0.150000

[2166 rows x 2 columns]


### Run the algorithm (write mode)
Then, use 'write' mode to write the results back to the Neo4j database.

In [9]:
result = neo4j.run("CALL gds.pageRank.write('interactions-all-books', { writeProperty: 'pagerank-all-books', relationshipWeightProperty: 'weight' })")

pprint.pprint(result.data())

[{'computeMillis': 42,
  'configuration': {'cacheWeights': False,
                    'concurrency': 4,
                    'dampingFactor': 0.85,
                    'maxIterations': 20,
                    'nodeLabels': ['*'],
                    'relationshipTypes': ['*'],
                    'relationshipWeightProperty': 'weight',
                    'sourceNodes': [],
                    'sudo': False,
                    'tolerance': 1e-07,
                    'writeConcurrency': 4,
                    'writeProperty': 'pagerank-all-books'},
  'createMillis': 0,
  'didConverge': True,
  'nodePropertiesWritten': 2166,
  'ranIterations': 15,
  'writeMillis': 2}]


## 4. Using graph features in ML
Now that we have the PageRank of each of the people in the Game of Thrones dataset, lets use it as a predictor variable.

We're going to predict whether a character will die based on `house`, `age`, `gender`, `culture`,  and `pageRank` of the corresponding node.

To prevent data leakage between the train/test sets, we'll need to calculate two different pageranks: 
- `pagerank_train` - the pagerank based on interactions in books 1-4.
- `pagerank_test` - the pagerank based on the interactions in book 5.

Then, we can train the model with only data derived from the first four books.

We already have the pagerank for the interactions in books 1 to 5 (the test data), so we quickly calculate them for books 1-4.


In [34]:
# Shorthand for everything we did in steps 2 & 3. This time, only look at interactions in the first four books.
neo4j.run("CALL gds.graph.exists($name) YIELD exists WHERE exists CALL gds.graph.drop($name) YIELD graphName RETURN graphName + ' was dropped.' as message", name = 'interactions-book-1-4')
neo4j.run("CALL gds.graph.create.cypher('interactions-book-1-4', 'MATCH (p) WHERE p:Person RETURN id(p) as id', 'MATCH (p)-[i:INTERACTS]->(p2:Person) WHERE i.book <= 4 RETURN id(p) AS source, i.weight as weight, id(p2) AS target') ")
neo4j.run("CALL gds.pageRank.write('interactions-book-1-4', { writeProperty: 'pagerank-book-1-4', relationshipWeightProperty: 'weight' })")

print("Training graph projection created and algorithm executed. Pagerank for interactions in books 1-4 written to graph.")

Training graph projection created and algorithm executed. Pagerank for interactions in books 1-4 written to graph.


### 4.1 Loading the features from Neo4j
Now that we have all the input data for our model, load data from Neo4j into a dataframe:

In [46]:
# Consider only the characters that have a defined age.
result = neo4j.run(
"""
MATCH (p:Person)
WHERE p.age >= 0
MATCH (p)-[:BELONGS_TO]->(h:House) 
OPTIONAL MATCH (p)-[:MEMBER_OF_CULTURE]->(c:Culture) 
OPTIONAL MATCH (p)-[:APPEARED_IN]->(b:Book)
RETURN p.name as name, p.age as age, p.gender as gender, 
       collect(c.name)[0] as culture, collect(h.name)[0] as house, 
       p.`pagerank-book-1-4` as pagerank_train,
       p.`pagerank-all-books` as pagerank_test, (p:Dead) as is_dead 
       ORDER BY pagerank_train DESC
""")

df = pd.DataFrame(result.data())
print(df[0:5])

               name  age  gender   culture          house  pagerank_train  \
0  Tyrion Lannister   32    male      None      Lannister       14.491704   
1   Tywin Lannister   58    male      None      Lannister        9.041645   
2       Sansa Stark   19  female  northmen          Stark        4.637840   
3       Walder Frey   97    male  rivermen           Frey        4.487632   
4     Samwell Tarly   22    male  westeros  Night's Watch        3.617952   

   pagerank_test  is_dead  
0      16.006423    False  
1       8.975134     True  
2       4.504770    False  
3       4.160120    False  
4       3.380056    False  


###  4.2 Process the data into a format the model expects
Now we transform the data into a format that the RandomForestClassifier can work with. This requires a couple of transformations:
- We convert our result variable (True/False) into (0/1).
- We create dummy variables for categorical values.
- We scale the numeric variables to a 0-1 range.

In [64]:
from sklearn.preprocessing import MinMaxScaler

# Select the right result columns. Make dummy variables, factorize non-numeric columns.
genders = pd.get_dummies(df['gender'])
cultures =  pd.get_dummies(df['culture'])
houses = pd.get_dummies(df['house'])
dead  = pd.DataFrame(pd.factorize(df['is_dead'])[0])
age = df['age']
pagerank_train = df['pagerank_train']
pagerank_test = df['pagerank_test']

# Construct the new data frame and scale.
data = pd.concat([houses, cultures, genders, age, pagerank_train, pagerank_test, dead], axis=1) 
data.columns = houses.columns.values.tolist() + cultures.columns.values.tolist()  + genders.columns.values.tolist() + ['age', 'pagerank_train', 'pagerank_test','is_dead']
scaler = MinMaxScaler() 
scaled_values = scaler.fit_transform(data) 
data.loc[:,:] = scaled_values

print(data[0:5])

   Arryn  Baratheon  Beesbury  Blackfyre  Blacks  Blackwood  Bolton  Bracken  \
0    0.0        0.0       0.0        0.0     0.0        0.0     0.0      0.0   
1    0.0        0.0       0.0        0.0     0.0        0.0     0.0      0.0   
2    0.0        0.0       0.0        0.0     0.0        0.0     0.0      0.0   
3    0.0        0.0       0.0        0.0     0.0        0.0     0.0      0.0   
4    0.0        0.0       0.0        0.0     0.0        0.0     0.0      0.0   

   Brax  Brotherhood Without Banners  ...  westerlands  westerman  westeros  \
0   0.0                          0.0  ...          0.0        0.0       0.0   
1   0.0                          0.0  ...          0.0        0.0       0.0   
2   0.0                          0.0  ...          0.0        0.0       0.0   
3   0.0                          0.0  ...          0.0        0.0       0.0   
4   0.0                          0.0  ...          0.0        0.0       1.0   

   wildlings  female  male   age  pagerank_t

### 4.3 Train/test split
Lets see how much we can boost the model accuracy by using `pagerank` as a feature.

We'll have to do a comparison - train two models  and compare their AUC scores:
1. A RandomForestClassifier that uses `[gender, house, age, culture]` to predict `is_dead`:
2. A RandomForestClassifier that uses `[gender, house, age, culture, pagerank]` to predict `is_dead`.

In [72]:
from sklearn.model_selection import train_test_split
y = data['is_dead']
y_train, y_test =  train_test_split(y, random_state=0, stratify=y, test_size=0.2)

# The original model uses [gender, house, age, culture] to predict [is_dead].
X = data.iloc[:, 0:len(data.columns)-3]
X_train = X.iloc[y_train.index]
X_test = X.iloc[y_test.index]

# The new model uses [gender, house, age, culture, pagerank] to predict [is_dead].
# The training data uses the pagerank for interactions in the first four books: [pagerank_train].
# The test data uses the pagerank for interactions in all books: [pagerank_test]
X2_train = data.iloc[:, 0:len(data.columns)-2]
X2_test = data.iloc[:, list(range(0,len(data.columns)-3)) + [len(data.columns)-2]]
X2_train = X2_train.iloc[y_train.index]
X2_test = X2_test.iloc[y_test.index]

### 4.4 Train and compare models

In [73]:
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(class_weight="balanced", n_estimators=10)
clf.fit(X_train, y_train)
print("Score without pagerank: ", roc_auc_score(y_test, y_score=clf.predict(X_test)))

clf.fit(X2_train, y_train)
print("Score with pagerank: ", roc_auc_score(y_test, y_score=clf.predict(X2_test)))

Score without pagerank:  0.6296526054590571
Score with pagerank:  0.716501240694789


We're seeing a slight improvement when adding our new feature! - however, keep in mind this dataset is *really tiny*: it's naturally very subceptible to randomness in the classifier and choice of train/test split. 

## What's next?
to learn more about the different execution modes of algorithms:
https://neo4j.com/docs/graph-data-science/current/common-usage/running-algos/

To speed up your process, consider looking into native projections:
https://neo4j.com/docs/graph-data-science/current/management-ops/native-projection/.

Read the docs on other algorithms, tips for modeling your data, and algo configurations:
https://neo4j.com/docs/graph-data-science/current/introduction/
